In [4]:
import requests
import json
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd


In [5]:
# wait
def wait(driver, selector_name):
    while (True):
        try:
            myElem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, selector_name)))
            if myElem is not None:
                break
        except TimeoutException:
            break

In [6]:
# scroll
def scroll(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [7]:
main_url = "https://soundcloud.com/discover"
set_link = []

In [8]:
import sys

def printProgressBar(i,max,postText):
    n_bar =10 #size of progress bar
    j= i/max
    sys.stdout.write('\r')
    sys.stdout.write(f"[{'=' * int(n_bar * j):{n_bar}s}] {int(100 * j)}%  {postText}")
    sys.stdout.flush()

In [9]:
# Drive the chrome to url link and return soup with html parser
def drive_to_url_page(driver, url):
    driver.get(url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [17]:
df = pd.read_csv (r'userHTML.csv', sep='delimiter', header=None)
print (df)

                                                      0
0     User ID\tUser Name\tUser Link\tPlaylist Count\...
1     133467077\tWestcoast Radio (EDM)\thttps://soun...
2     214279944\tIlyana Azman\thttps://soundcloud.co...
3     79816260\tAustin24heck\thttps://soundcloud.com...
4     11164017\tUlana\thttps://soundcloud.com/itsula...
...                                                 ...
997   58066195\tDawon  Chung\thttps://soundcloud.com...
998   453887553\tTerezTéro808\thttps://soundcloud.co...
999   589586403\tHayat\thttps://soundcloud.com/cafem...
1000  329713478\tReb Izzle\thttps://soundcloud.com/r...
1001  420049806\tMitzi Martini\thttps://soundcloud.c...

[1002 rows x 1 columns]


C:\Users\DELL\AppData\Local\Temp/ipykernel_10028/434330539.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv (r'userHTML.csv', sep='delimiter', header=None)


In [33]:
set_user = [i.split('\t')[2] for i in df[0]]

In [34]:
set_user = set_user[1:]

In [46]:
set_link1 = set_user[:100]
set_link2 = set_user[100:200]
set_link3 = set_user[200:300]
set_link4 = set_user[300:400]
set_link5 = set_user[400:500]
set_link6 = set_user[500:600]
set_link7 = set_user[600:700]
set_link8 = set_user[700:800]
set_link9 = set_user[800:900]
set_link10 = set_user[900:]

In [47]:
def getTracks(set_user):
    driver = webdriver.Chrome()
    set_tracks = []
    for i, item in enumerate(set_user):
        temp = item + "/tracks"
        driver.get(temp)

        wait(driver, selector_name='infoStats__value sc-font-light')

        scroll(driver)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        user_id = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
        link_tracks = soup.find_all('a', {'class': 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
        link_tracks = link_tracks[:-3]
        info_start = soup.find_all("div", {"class": "infoStats__value sc-font-light"})
        track_count = info_start[2].getText()
        if track_count != '0':
            for link in link_tracks:
                string_temp = 'https://soundcloud.com'
                string_temp = string_temp + link['href']
                driver.get(string_temp)
                wait(driver, selector_name='sc-ministats-item')
                soup = BeautifulSoup(driver.page_source, "html.parser")
                id_track = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
                name_track = soup.find('h1', {'class': 'soundTitle__title sc-font g-type-shrinkwrap-inline g-type-shrinkwrap-large-primary theme-dark'}).getText().strip()
                time_release_track = soup.find('time', {'class': 'relativeTime'})['title']
                info_track = soup.find_all('span', {'aria-hidden': 'true'})
                plays = info_track[1].getText()
                likes = info_track[2].getText()
                reposts = info_track[3].getText()

                tup = [user_id, id_track, name_track, time_release_track, plays, likes, reposts]
                set_tracks.append(tup)
        else:
            set_tracks.append([user_id, "None", "None", "None", "None", "None", "None"])
        
        printProgressBar(i, len(set_user), "'%'")
    driver.close()
    return set_tracks

In [48]:
set_tracks = getTracks(set_link1)
len(set_tracks)

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(set_tracks, columns=['User ID',
                                      'Tracks ID',
                                      'Tracks Name',
                                      'Time_Release_Track',
                                      'Plays',
                                      'Likes',
                                      'Repost'])
df.to_csv('tracksHTML1.csv', index=False, sep='\t', encoding='utf-8')

In [ ]:
def getTracksFromPlaylists(playlist_url, optional):
    driver = webdriver.Chrome()
    driver.get(playlist_url)
    wait(driver, selector_name='trackItem__trackTitle')
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    user_name = soup.find("a", {"class": "userBadge__usernameLink"}).getText().strip()
    playlist_title = soup.find("h1", {"class": "soundTitle__title"}).getText().strip()
    num_of_track = soup.find("div", {"class": "genericTrackCount__title"}).getText()
    create_date = soup.find("time", {"class": "relativeTime"})["title"]
    datas = soup.find_all("span", {"aria-hidden": "true"})

    likes = datas[1].getText()
    reports = datas[2].getText()

    track_list = []
    track_str = []

    if optional == 0:
        track_items = soup.find_all("a", {"class": "trackItem__trackTitle sc-link-dark sc-link-primary sc-font-light"})
        for track_item in track_items:
            track_name = 'https://soundcloud.com' + track_item['href']
            track_list.append(track_name.strip().replace("\n", " "))
    elif optional == 1:
        track_items = soup.find_all("a", {"class": "trackItem__trackTitle"})
        track_urls = []

        for track_item in track_items:
            track_urls.append('https://soundcloud.com' + track_item['href'])

        for url in track_urls:
            driver.get(url)
            wait(driver, selector_name='soundTitle__usernameTitleContainer')
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            meta = soup.find("meta", property="twitter:app:url:iphone")
            track_list.append(meta["content"].split(':')[-1])

    track_str = ','.join(track_list)
    if track_str == '':
        track_str = 'None'
    
    driver.close()
    return [user_name, playlist_title, num_of_track, create_date, likes, reports, track_str]

In [ ]:
def getPlaylistURLs(user_playlist_url):
    driver = webdriver.Chrome()
    driver.get(user_playlist_url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    contents = soup.find_all("a", {"class": "sound__coverArt"})
    urls = []

    for content in contents:
        urls.append('https://soundcloud.com' + content['href'])
    
    driver.close()
    return urls

In [ ]:
def getPlaylists(user_playlist_urls, optional):
    playlist_lst = []
    for i, user_url in enumerate(user_playlist_urls):
        driver = webdriver.Chrome()
        driver.get(user_url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        user_id = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
        driver.close()
        playlist_urls = getPlaylistURLs(user_url + '/sets')
        
        for j, playlist_url in enumerate(playlist_urls):
            playlist_lst.append([user_id] + getTracksFromPlaylists(playlist_url, optional))
            printProgressBar(j, len(playlist_urls), '')
        
        printProgressBar(i, len(user_playlist_urls), "'%'")
    return playlist_lst

In [ ]:
set_playlists = getPlaylists(set_link1, optional=0)
len(set_playlists)

In [ ]:
df = pd.DataFrame(set_playlists, columns=['User ID',
                                          'User name',
                                          'Playlist name',
                                          'Track count',
                                          'Create at',
                                          'Likes count',
                                          'Reposts count',
                                          'Track list'
                                         ])
df.to_csv('playlistHTML1.csv', index=False, sep='\t', encoding='utf-8')